# Page Rank

## Read Files

In [1]:
path_name_documents = './Databases/prova/prova50.jsonl'



In [13]:
import json
import numpy as np
import string

def readFile(path_name):
    # Load the JSONL file into a list
    with open(path_name, 'r') as f:
        lines = f.readlines()

    # Convert each JSON object into a dictionary
    dicts = [json.loads(line) for line in lines]

    # Convert the dictionaries into arrays and stack them vertically
    arrays = np.vstack([np.array(list(d.values())) for d in dicts])

    # Convert the arrays into a list of lists
    text = arrays.tolist()
    
    return text

documents = readFile(path_name_documents)
print(documents)


[['ug7v899j', 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with

# Sparse Vectors

## TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


def calculateTFIDF(documents):
    
    vectorizer = TfidfVectorizer()
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(for doc in documents)

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
    
        

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(documents)

SyntaxError: invalid syntax (3603363790.py, line 8)

## CosineSimilarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity


def similarity(tfidf_matrix_docs):
    # calcoliamo la cosine similarity tra i documenti
    cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)

    print(cos_sim)

#similarity(tfidf_matrix_docs)

In [12]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_documents_from_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    docs = [doc['text'] for doc in data['documents']]
    return docs, data

doc = load_documents_from_json(documents)

print(doc)

TypeError: unhashable type: 'list'

In [ ]:
def create_similarity_matrix(docs):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
    cos_sim = cosine_similarity(tfidf_matrix)
    return cos_sim



In [ ]:
def group_similar_documents(similarity_matrix, threshold):
    num_docs = len(similarity_matrix)
    similar_docs = {i: set() for i in range(num_docs)}
    for i in range(num_docs):
        for j in range(i+1, num_docs):
            if similarity_matrix[i, j] >= threshold:
                similar_docs[i].add(j)
                similar_docs[j].add(i)
    groups = []
    seen = set()
    for i in range(num_docs):
        if i not in seen:
            group = {i} | similar_docs[i]
            groups.append(group)
            seen |= group
    return groups



In [ ]:
def find_similar_docs(path_name_documents, threshold):
    docs, data = load_documents_from_json(path_name_documents)
    cos_sim = create_similarity_matrix(docs)
    groups = group_similar_documents(cos_sim, threshold)
    result = {}
    for i, group in enumerate(groups):
        if len(group) > 1:
            result[f'group_{i+1}'] = [data['documents'][j] for j in group]
    return result

find_similar_docs(path_name_documents, '0.5')